In [ ]:
# (Run this/run all) Import dependencies
import os
import re

In [ ]:
# (First time only) Clone AUTOMATIC1111 SD WebUI repository
if os.path.exists('/notebooks/stable-diffusion-webui'):
    print("Clone already completed. No action needed.")
else:
    %cd /notebooks
    !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui

In [ ]:
# (Run this) Update repositories
%cd /notebooks/stable-diffusion-webui
!git checkout master
!git pull

%cd /notebooks/paperspace-automatic1111
!git checkout master
!git pull

In [ ]:
# (Run this) The function to download files (Currently only support Huggingface URLs)
def download_files(file_urls, download_path, file_type):
    for file_url in file_urls:
        segments = file_url.split('/')
        file_name = segments[-1] # Todo: Add support for non-huggingface URLs
        new_file_name = file_name
        
        if file_type == 'VAE':
            split_segments = [segment.split('.') for segment in segments]
            file_format = split_segments[-1]
            if file_format[-1] == 'ckpt':
                version = re.search(r'\d+', file_format[0]).group()
                new_file_name = re.sub(r'\d+', version, file_format[0]) + '.ckpt.vae.pt'
        
        if not os.path.exists(os.path.join(download_path, new_file_name)):
            !wget -P $download_path $file_url
            if (file_type == 'Model' || file_type == 'VAE') && new_file_name != file_name:
                os.rename(os.path.join(download_path, file_name), os.path.join(download_path, new_file_name))
            print(f'{file_type} {file_name} downloaded.')
        else:
            print(f'{file_type} {file_name} already exists. No action needed.')

In [ ]:
# (Run this) Input URLS to download supporting files
model_urls = [
    'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_sfw.safetensors',
    # Add more if needed
]

vae_urls = [
    'https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt',
    # Add more if needed
]

lora_urls = [
    'https://civitai.com/api/download/models/75069',
    # Add more if needed
]

# Text Inversion
embedding_urls = [
    'https://civitai.com/api/download/models/74332',
    # Add more if needed
]

In [ ]:
# (Run this) Locate the download path
model_download_path = '/notebooks/stable-diffusion-webui/models/Stable-diffusion'
vae_download_path = '/notebooks/stable-diffusion-webui/models/VAE'
lora_download_path = '/notebooks/stable-diffusion-webui/models/Lora'
embedding_download_path = '/notebooks/stable-diffusion-webui/embeddings'

In [ ]:
# (Run this) Download the Models
download_files(model_urls, model_download_path, 'Model')

In [ ]:
# (Run this) Download the VAE
download_files(vae_urls, vae_download_path, 'VAE')

In [ ]:
# (Run this) Download the Lora
download_files(lora_urls, lora_download_path, 'Lora')

In [ ]:
# (Run this) Download the Embedding/Text Inversion
download_files(embedding_urls, embedding_download_path, 'Embedding/Text Inversion')

In [ ]:
### (Run this) Launching public URL. wait until PUBLIC URL appeared in the output, then ctrl+click on the URL
# When asked for credentials, fill it with => (username = admin), (password = admin).

# Solve dependency issue
!pip install --upgrade torch==1.9.0
!pip install --upgrade torchvision==0.10.0
!pip install --upgrade torchaudio==0.9.0

# Launch
%cd /notebooks/stable-diffusion-webui
!python3 launch.py --enable-insecure-extension-access --share --gradio-auth admin:admin --precision full --no-half --xformers --disable-safe-unpickle